In [ ]:
'''
Porfolio Project By Muhammad Haseeb Farooq
Contact: hf.haseebfarooq@gmail.com
'''
#import packages
import os
os.chdir("/content/drive/MyDrive")
Dataset="/content/drive/MyDrive/data"
Data_Dir=os.listdir(Dataset)
print(Data_Dir)

In [ ]:
#import necessary libraries
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

img_row, img_col = 112, 112

images = []
labels = []

for category in Data_Dir:
  folder_path = os.path.join(Dataset, category)
  for img in os.listdir(folder_path):
    img_path = os.path.join(folder_path, img)
    img = cv2.imread(img_path)

    try:
      #converting the image into gray scale
      grayscale_img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
      
      #Resizing the gray scaled images into the size 56 x 56 in order to keep size of the image consistant
      resized_img=cv2.resize(grayscale_img,(img_row, img_col))
      images.append(resized_img)
      labels.append(category)
  #Exception Handling in case any error occur
    except Exception as e:
      print("Exception:",e)

images=np.array(images)/255.0
images=np.reshape(images,(images.shape[0],img_row, img_col,1)) 

#perform one hot encoding on the labels since the label are textual form
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)
labels = np.array(labels)

(train_X, test_X, train_y, test_y) = train_test_split(images, labels, test_size=0.25, random_state=0)



In [ ]:
#Building CNN
#import necessary keras/TensorFlow libraries
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D

#Define model parameters
num_classes = 2
batch_size = 32

#Build CNN Model Layers using Sequential API
model = Sequential()

#First Layer Group containing Convolution, Relu and MaxPooling layers
model.add(Conv2D(64,(3,3),input_shape=(img_row, img_col, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#Second layer Group containing Convolution, Relu and MAXPooling
model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#Flatten and Dropout layers to stack output
model.add(Flatten())
model.add(Dropout(0.5))

#Softmax Classifier
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes,activation='softmax'))

#printing model summary
print(model.summary())


In [ ]:
#plot model structure
from keras.utils.vis_utils import plot_model
plot_model(model, to_file="face_mask_model_structure.png")

In [ ]:
#train model
from keras.optimizers import Adam
epochs = 50

model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics = ['accuracy'])

fitted_model = model.fit(
    train_X,
    train_y,
    epochs = epochs,
    validation_split=0.25)

In [ ]:
#ploting the training result
from matplotlib import pyplot as plt
#Plotting Training and VAlidation loss
plt.plot(fitted_model.history['loss'], label='training loss')
plt.plot(fitted_model.history['val_loss'], label='Val loss')
plt.xlabel('Number of Epochs')
plt.ylabel('loss values')
plt.legend()
plt.show()

#plot the training and validation accuracy
plt.plot(fitted_model.history['accuracy'], label='training acc')
plt.plot(fitted_model.history['val_accuracy'], label='Val acc')
plt.xlabel('Number of Epochs')
plt.ylabel('acc values')
plt.legend()
plt.show()



In [ ]:
#saving my model with name
model.save('face_mask_model.h5')